In [96]:
import pandas as pd
import nltk

from nltk.stem import PorterStemmer

file = "verification results.csv"

df = pd.read_csv(file, encoding='utf-8')

# only non-relevant
df['relevant'] = df.relevant.str.lower()
df_rel = df[df['relevant'] == 'yes']
df_notrel= df[df['relevant'] != 'yes']

df_notrel.head()

,Unnamed: 0,text,scrape_time,tweet_id,created_at,reply_count,quote_count,like_count,retweet_count,geo_full_name,...,num_followers,search_keywords,search_neighbourhood,sentiment,score,score_matches,relevant,File #,From,Notes
0,1150.0,First time seeing snow in about 15+ years! 20 ...,34:31.7,1.54E+18,2022-06-19 15:17:50+00:00,0,0,0,0,NaN,...,12736,"Victoria (boys ""and"" girls club south vanouver...",Victoria,Neutral,0.584249,Yes,no,0,Josh,NaN
1,1277.0,"So, taxpayers are flying you &amp; your entour...",34:33.3,1.54E+18,2022-06-15 16:44:20+00:00,0,0,4,1,NaN,...,1,Victoria (burnside gorge neighbourhood associa...,Victoria,Negative,0.799547,Yes,no,0,Josh,NaN
2,123.0,If anybody on the planet deserves to get paid ...,32:27.0,1.54E+18,2022-06-18 17:52:40+00:00,0,0,4,0,NaN,...,178,(sooke OR gonzales OR foul bay OR the gorge OR...,sooke OR gonzales OR foul bay OR the gorge OR ...,Negative,0.807167,Yes,no,0,Josh,NaN
3,289.0,@tomhunt1988 So this â€˜eliteâ€™ is not allowe...,32:40.3,1.54E+18,2022-06-15 13:36:35+00:00,0,0,0,0,NaN,...,212,(view royal OR harris green OR hillside-quadra...,view royal OR harris green OR hillside-quadra ...,Neutral,0.560878,No,no,0,Josh,NaN
4,597.0,Can't wait for tennis camp tomorrow! Reminder:...,33:41.9,1.54E+18,2022-06-19 12:57:06+00:00,0,0,8,1,NaN,...,217,(saanich OR uplands OR quadra village OR centr...,saanich OR uplands OR quadra village OR centra...,Positive,0.933474,Yes,no,0,Josh,NaN


In [172]:
len(df)
len(df_rel)

55

# Non-relevant tweets

In [173]:
def parse_df(df):
    ### Prep our corpus
    corpus = ' '.join(df['text'])
    corpus = corpus.lower()

    ### remove specific words
    # amp = &amp;
    # https = url
    corpus = corpus.replace("amp", "").replace("https","")

    # tokenize
    words = nltk.tokenize.word_tokenize(corpus)

    # stem
    # non characters out
    ps = PorterStemmer()

    words = [w for w in words if w.isalpha()]

    ## count
    word_dict = nltk.FreqDist(words)

    sw = nltk.corpus.stopwords.words('english')

    words_except_sw = nltk.FreqDist(w for w in words if w not in sw)

    words_except_sw
    #corpus[:1000]

    df_prime = pd.DataFrame(words_except_sw.most_common(10), columns=['Word', 'Frequency'])

    df_prime['Frequency_%'] = df_prime['Frequency'] / len(df)

    return df_prime

parse_df(df_notrel)

,Word,Frequency,Frequency_%
0,victoria,74,0.459627
1,park,26,0.161491
2,c,19,0.118012
3,central,17,0.105590
4,gorge,15,0.093168
5,cing,12,0.074534
6,one,12,0.074534
7,get,12,0.074534
8,day,12,0.074534
9,crime,12,0.074534


# Relevant Tweets

In [174]:
parse_df(df_rel)

,Word,Frequency,Frequency_%
0,housing,25,0.454545
1,victoria,21,0.381818
2,yyj,12,0.218182
3,people,11,0.200000
4,violence,10,0.181818
5,would,9,0.163636
6,langford,7,0.127273
7,like,7,0.127273
8,homeless,7,0.127273
9,city,6,0.109091


# Most used search words (not relevant)

In [175]:

def top_keywords(this_df):

    second_count = {}
    first_count = {}

    for index, row in this_df.iterrows():
        sk = row['search_keywords'].strip()
        sk.replace('"and"', "and")

        ### build words to search
        dual = True if sk[0] == "(" else False 
        
        if dual:
            split = sk.split(") (")

            ### grab first 
            first = split[0].strip().replace("(", "").split("OR")
            first = [f.strip().lower() for f in first]

            
        else:
            split = sk.split("(")

            first = [split[0].strip().lower()]

        ### first
        print(first)

        ### grab part 2 keywords
        second = split[1].strip()
        second = second[:second.find(")")]
        second = [s.strip().lower() for s in second.split("OR")] 

        ### start counting -> Keywords
        text = row['text'].strip().lower()

        for s in second:

            ### add to dict
            if s not in second_count:
                second_count[s] = 0

            ### check if it in
            if s in text:
                second_count[s] += 1
                continue 

            ### break it apart
            Found = False 
            for sub_s in s.split():
                if sub_s not in text:
                    ### no point continuing if one word not in 
                    Found = False 
                    break 
                else:
                    Found = True

            if Found:
                second_count[s] += 1

        ### deal with first
        for f in first:

            # if starts with 'from:'
            # we are dealing with usernames not locations
            if f.startswith('from:'):
                pass

            else:

    df_sc = pd.DataFrame.from_dict(second_count, orient='index', columns=['count'])
    df_sc['count_p'] = df_sc['count'] / len(this_df)
    
    return df_sc.astype(float).nlargest(10, 'count_p')
    
top_keywords(df_notrel)

['victoria']
['victoria']
['sooke', 'gonzales', 'foul bay', 'the gorge', 'royal athletic park', 'selkirk trestle', 'wsã¡neä‡']
['view royal', 'harris green', 'hillside-quadra', 'vic west', 'stadacona park', 'lochside', 'tsartlip']
['saanich', 'uplands', 'quadra village', 'central park', 'point ellice park', 'nuu-chah-nulth', 'malahat']
['victoria']
['victoria']
['saanich', 'uplands', 'quadra village', 'central park', 'point ellice park', 'nuu-chah-nulth', 'malahat']
['victoria']
['sooke', 'gonzales', 'foul bay', 'the gorge', 'royal athletic park', 'selkirk trestle', 'wsã¡neä‡']
['view royal', 'harris green', 'hillside-quadra', 'vic west', 'stadacona park', 'lochside', 'tsartlip']
['saanich', 'uplands', 'quadra village', 'central park', 'point ellice park', 'nuu-chah-nulth', 'malahat']
['saanich', 'uplands', 'quadra village', 'central park', 'point ellice park', 'nuu-chah-nulth', 'malahat']
['sooke', 'gonzales', 'foul bay', 'the gorge', 'royal athletic park', 'selkirk trestle', 'wsã¡neä

,count,count_p
camp,16.0,0.099379
camping,13.0,0.080745
poor,11.0,0.068323
crime,11.0,0.068323
violence,9.0,0.055901
victoria ready,8.0,0.049689
city of victoria,7.0,0.043478
housing,7.0,0.043478
tent,6.0,0.037267
ops,5.0,0.031056


### Most Common (relevant)

In [170]:
top_keywords(df_rel)

,count,count_p
housing,15.0,0.272727
violence,10.0,0.181818
homeless,5.0,0.090909
avi,4.0,0.072727
healthcare,3.0,0.054545
bc housing,2.0,0.036364
affordable housing,2.0,0.036364
affordable,2.0,0.036364
low income,1.0,0.018182
vibrant victoria,1.0,0.018182
